In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np
import numpy.linalg as la
from scipy.integrate import RK45

import sympy as sym

from table_maker import *
from functools import partial
from itertools import *
from math import ceil

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size': 16})

# Kilpatrick et al SIADS 2008

$$\begin{align*}
    \tau_m u_t &= -u + \int_{-\infty}^{\infty} w(x,x^\prime) f( u(x^\prime,t) ) dx^\prime - \beta v \\
    \frac{1}{\alpha} v_t &= -v + u
\end{align*}$$

In [48]:
a, b = -10, 200
n = 10**3

tau_m = 1

#alpha = .1
# beta = 2.5
alpha, beta = 0.04, 2.0

xs = np.linspace(a,b,n)
h = xs[1]-xs[0]

def firing_rate(u, theta=.2, eta=20):
#     return 1/(1 + np.exp(-eta*(u-theta)))
    return np.heaviside(u-theta, 0)
    
M = .5*np.exp( - np.abs( np.subtract.outer(xs, xs) ) ) * h

def convolution(u):
    return M@firing_rate(u)

def RK4_step(F, t, u, dt):
    k1 = F(t,u)
    k2 = F(t+dt/2, u + dt/2*k1)
    k3 = F(t+dt/2, u + dt/2*k2)
    k4 = F(t+dt, u + dt*k3)
    return u + dt/6*(k1+2*k2+2*k3+k4)

F = lambda t, u: np.array([1/tau_m * (-u[0] + convolution(u[0]) - beta*u[1]),
                           alpha * (-u[1] + u[0]) ])

In [49]:
def cos_bell(x, center=0, width=2*np.pi, height=1):
    return (np.cos((x-center)/width*2*np.pi)+1)/2*height * np.heaviside(x-center+width/2,0) * np.heaviside(-x+center+width/2,0)

In [50]:
# Cosine bell with no negative feedback
# u0 = np.zeros((2,n))
# u0[0,np.abs(xs)<np.pi] = (np.cos( xs[np.abs(xs)<np.pi] ) + 1)/2

# Cosine bells, offset
# u0 = np.zeros((2,n))
# u0[0,np.abs(xs)<np.pi] = (np.cos( xs[np.abs(xs)<np.pi] ) + 1)/2
# shift = 2*np.pi
# u0[1,np.abs(xs+shift)<np.pi] = (np.cos( xs[np.abs(xs+shift)<np.pi]+shift ) + 1)/2

# Cosine Bell, with heaviside feedback
# u0 = np.zeros((2,n))
# u0[0,np.abs(xs)<np.pi] = (np.cos( xs[np.abs(xs)<np.pi] ) + 1)/2
# u0[1] = np.heaviside(-xs, 0) - np.heaviside(-np.pi-xs,0)

# test 3
# u0 = np.zeros((2,n))
# u0[0,np.abs(xs+20)<np.pi] = (np.cos( xs[np.abs(xs+20)<np.pi] ) + 1)/2 * (xs[np.abs(xs+20)<np.pi]+20)
# shift = np.pi/2 + 20
# u0[1,np.abs(xs+shift)<np.pi] = (np.cos( xs[np.abs(xs+shift)<np.pi]+shift ) + 1)/2

# test 4
u0 = np.zeros((2,n))
u0[0] = cos_bell(xs, center=0)
u0[1] = cos_bell(xs, center = -np.pi)



t0 = 0
t_final = 100
k = 1e-2

########################
steps = int(np.ceil( (t_final - t0)/k ))
k = (t_final - t0)/steps

us = [u0]
ts = [t0]

for step in range(steps):
    ts += [ts[-1] + k]
    us += [ RK4_step(F, ts[-1], us[-1], k) ]
    print('step %d/%d' % (step,steps), end='\r')


In [51]:
stride = 30

fig, ax = plt.subplots(1, 1, figsize=(12,8))

line_u, = ax.plot(xs, us[0][0], 'b-', label="$u$")
line_v, = ax.plot(xs, us[0][1], 'g-', label="$v$")
y_min = np.min(us)
y_max = np.max(us)
y_min -= .05*np.abs(y_max - y_min)
y_max += .05*np.abs(y_max - y_min)
window = y_min, y_max

ax.legend(loc='upper right')
ax.set_ylim(*window)
ax.set_xlim(a, b)
ax.legend()

def animate(i):
    print('step %d/%d' % (i,len(ts)), end='\r')
    line_u.set_ydata(us[i][0])
    line_v.set_ydata(us[i][1])
    return line_u,


# Init only required for blitting to give a clean slate.
def init():
    line_u.set_ydata(us[0])
    return line_u,

anim = animation.FuncAnimation(fig, animate, np.arange(0,len(ts),stride), init_func=init,
                              interval=1/24*1000, blit=True)

# anim.save('PE_pulse.mp4', writer='imagemagick', fps=24)

plt.close()
HTML(anim.to_html5_video())